In [1]:
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
gamma = 0.99 # discount factor
seed  = 543
render = False
log_interval = 10

env = gym.make('CartPole-v1')
env.seed(seed)
torch.manual_seed(seed)

In [3]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

In [4]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()

In [8]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()

def sim_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    actions = policy(state)
    _, action = actions.max(1)
    return action.item()

def finish_episode():
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

def sim():
    state = env.reset()
    for t in range(1, 10000):
        action = sim_action(state)
        state, _, _, _ = env.step(action)
        env.render()

In [6]:
running_reward = 10
for i_episode in count(1):
    state, ep_reward = env.reset(), 0
    for t in range(1, 10000):  # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        policy.rewards.append(reward)
        ep_reward += reward
        if done:
            break

    running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
              i_episode, ep_reward, running_reward))
        #print(action)
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        break

Episode 10	Last reward: 26.00	Average reward: 16.00
Episode 20	Last reward: 16.00	Average reward: 14.85
Episode 30	Last reward: 49.00	Average reward: 20.77
Episode 40	Last reward: 45.00	Average reward: 27.37
Episode 50	Last reward: 44.00	Average reward: 30.80
Episode 60	Last reward: 111.00	Average reward: 42.69
Episode 70	Last reward: 131.00	Average reward: 70.39
Episode 80	Last reward: 87.00	Average reward: 76.68
Episode 90	Last reward: 97.00	Average reward: 96.58
Episode 100	Last reward: 87.00	Average reward: 95.25
Episode 110	Last reward: 338.00	Average reward: 110.90
Episode 120	Last reward: 500.00	Average reward: 166.03
Episode 130	Last reward: 169.00	Average reward: 178.34
Episode 140	Last reward: 124.00	Average reward: 179.94
Episode 150	Last reward: 111.00	Average reward: 168.89
Episode 160	Last reward: 32.00	Average reward: 154.12
Episode 170	Last reward: 193.00	Average reward: 171.36
Episode 180	Last reward: 340.00	Average reward: 186.44
Episode 190	Last reward: 93.00	Average

In [16]:
sim()

KeyboardInterrupt: 

In [ ]:
torch.save(policy.state_dict(), "./models/model1_500.pth")